In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import pandas as pd
import glob
import spateo as st
from joblib import Parallel, delayed
from functools import partial
from scipy import ndimage as ndi
from scipy.stats import median_abs_deviation

import os
os.chdir('/f_active/paper_23/') # laune

2023-08-19 19:54:56.573189: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 19:54:56.739796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 19:54:57.306432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/peter/mambaforge/envs/spateo/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:38: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently Fa

In [2]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [3]:
masks = {x.split('/')[-1].split('_mask.npy')[0]:x for x in glob.glob('input/01_stereoseq_preprocessing/segmentation/masks/*npy')}
gems = {x.split('/')[-1].split('.tissue')[0]:x for x in glob.glob('input/01_stereoseq_preprocessing/gem/**/**/*gem.gz', recursive=True)}
spateo_in_files = {x.split('/')[-1].split('.counts')[0]:x for x in glob.glob('input/01_stereoseq_preprocessing/gem/**/**/*spateo.csv.gz', recursive=True)}

In [4]:
def mask_to_adata(k):
    adata = st.io.read_bgi_agg(spateo_in_files[k])
    mask = np.load(masks[k])
    labeled_array, num_features = ndi.label(mask)
    adata.layers['labeled_array'] = labeled_array
    cell_adata = st.io.read_bgi(
        gems[k],
        segmentation_adata=adata,
        labels_layer='labeled_array',
    )
    cell_adata = cell_adata[cell_adata.obs['area'] > 10]
    cell_adata = cell_adata[-is_outlier(cell_adata, 'area', 5),].copy()
    print(cell_adata.shape)
    output_path = 'input/01_stereoseq_preprocessing/adata'
    if not os.path.exists(output_path):
        print("Output path does not exist. Creating...")
        os.makedirs(output_path)
    cell_adata.write_h5ad(output_path + '/' + k + '.h5ad')

In [5]:
spateo_in_files

{'FP200000578TR_C5': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000578TR_C5.counts_spateo.csv.gz',
 'FP200000578TR_C3': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000578TR_C3.counts_spateo.csv.gz',
 'FP200000542BR_F3': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000542BR_F3.counts_spateo.csv.gz',
 'FP200000542BR_A5': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000542BR_A5.counts_spateo.csv.gz',
 'FP200000542BR_F4': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000542BR_F4.counts_spateo.csv.gz',
 'FP200000542BR_A3': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000542BR_A3.counts_spateo.csv.gz',
 'FP200000542BR_A1': 'input/01_stereoseq_preprocessing/gem/batch1/v2/FP200000542BR_A1.counts_spateo.csv.gz',
 'D01066E6': 'input/01_stereoseq_preprocessing/gem/batch2/D01066E6.counts_spateo.csv.gz',
 'SS200000844BL_C6': 'input/01_stereoseq_preprocessing/gem/batch2/SS200000844BL_C6.counts_spateo.csv.gz',
 'SS200000953BR_C5': 'input/01_stereoseq_

In [10]:
k = 'FP200000578TR_C5'
adata = st.io.read_bgi_agg(spateo_in_files[k])
mask = np.load(masks[k])
labeled_array, num_features = ndi.label(mask)
adata.layers['labeled_array'] = labeled_array
cell_adata = st.io.read_bgi(
    gems[k],
    segmentation_adata=adata,
    labels_layer='labeled_array',
    add_props = True
)
cell_adata = cell_adata[cell_adata.obs['area'] > 10]
cell_adata = cell_adata[-is_outlier(cell_adata, 'area', 5),].copy()

|-----> Constructing count matrices.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> <select> labeled_array layer in AnnData Object
|-----> Using labels provided with `segmentation_adata` and `labels_layer` arguments.
|-----> Constructing count matrices.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.


/home/peter/mambaforge/envs/spateo/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



In [9]:
adata.layers['labeled_array']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [11]:
cell_adata.obs

,area
1,810.0
2,457.0
3,329.0
4,797.0
5,839.0
...,...
31880,193.0
31881,913.0
31882,228.0
31883,452.0


In [6]:
Parallel(n_jobs=4)(delayed(mask_to_adata)(sn) for sn in masks.keys())

2023-08-17 18:16:59.295205: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 18:16:59.295206: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 18:16:59.295206: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-17 18:16:59.308349: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly 

|-----> Constructing count matrices.
|-----> Constructing count matrices.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> Constructing count matrices.
|-----> Constructing count matrices.
|-----> <select> labeled_array layer in AnnData Object
|-----> Using labels provided with `segmentation_adata` and `labels_layer` arguments.
|-----> <select> labeled_array layer in AnnData Object
|-----> Using labels provided with `segmentation_adata` and `labels_layer` arguments.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns i

/home/peter/mambaforge/envs/spateo/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> Constructing count matrices.


/home/peter/mambaforge/envs/spateo/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



...

|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
|-----> <select> labeled_array layer in AnnData Object
|-----> Using labels provided with `segmentation_adata` and `labels_layer` arguments.
|-----> Constructing count matrices.


/home/peter/mambaforge/envs/spateo/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning:

Transforming to str index.



|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

(32289, 25514)
